In [33]:
import sys
sys.path.append('/Users/chrisolen/Documents/uchicago_courses/optimization/project/urban-demand-allocation')

import pandas as pd
import numpy as np
import json

from py2neo import Graph

from sklearn.linear_model import LinearRegression

import utilities
from demand_models.filter_business_data import business_filter

In [34]:
df_types = pd.read_csv('../../data/dtypes.csv')['dtypes']
bus = pd.read_csv("../../data/chi_bus_cleaned.csv",dtype=df_types.to_dict())

In [35]:
years = [2016,2017,2015]
years.sort(reverse=True)
naics = ['445110','335']



In [36]:
demand = business_filter(bus, years, naics)

In [37]:
demand

,abi,year,employee_size_location,sales_volume_location,year_2016,employee_size_location_2016,sales_volume_location_2016,year_2015,employee_size_location_2015,sales_volume_location_2015,year_established,latitude,longitude
0,5780192,2017,45.0,7959.0,2016.0,45.0,12148.0,2015.0,60.0,16198.0,1953.0,41.96843,-87.73453
1,8967192,2017,40.0,9640.0,2016.0,40.0,13420.0,2015.0,40.0,13420.0,1925.0,41.88798,-87.67486
2,9321290,2017,25.0,3853.0,2016.0,25.0,6089.0,2015.0,25.0,3000.0,1940.0,41.86212,-87.69216
3,9323510,2017,20.0,4287.0,2016.0,20.0,6045.0,2015.0,20.0,6045.0,1917.0,41.88546,-87.73210
4,9324856,2017,38.0,9063.0,2016.0,38.0,12638.0,2015.0,38.0,12638.0,1938.0,41.94774,-87.64933
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,418290722,2017,52.0,12218.0,2016.0,52.0,19176.0,2015.0,52.0,19176.0,1925.0,41.77312,-87.75264
887,533855128,2017,6.0,1410.0,2016.0,6.0,2212.0,2015.0,6.0,2212.0,1917.0,41.82636,-87.73097
888,655861375,2017,500.0,0.0,2016.0,500.0,NaN,2015.0,500.0,NaN,1917.0,41.88118,-87.63913
889,410866459,2017,3.0,767.0,2016.0,3.0,796.0,2015.0,5.0,1327.0,NaN,41.88171,-87.63086


In [38]:
# Pulling out coordinates for each store

business_coordinates = list(zip(demand["longitude"],demand["latitude"]))

# Pulling neighborhood polygons
with open('../../data/geo_shape_files/neighborhood_reformatted.json','r') as f:
    neighborhoods = json.load(f)

In [39]:
# Connect to graph db

uri = "bolt://localhost:7687"
graph = Graph(uri, auth=("neo4j", "password"))



In [40]:
def graph_to_demand_model(graph, demand_frame, feature, business_coordinates, locality_type):
    
    # Loading average property value for the neighborhood in which the stores is located
    demand_frame[feature] = np.nan

    outside_search_area = []

    for i in range(len(business_coordinates)):

        point_location = utilities.point_lookup(locality_type,business_coordinates[i])
    
        try:
            result = float(dict(pd.DataFrame(graph.run('match (a:neighborhood) \
                                                    where a.name = "{}" return a'.format(point_location)). \
                                                    to_table()).iloc[0,0])[feature])
            ## coordinates and df indices should be the same ## 
            demand_frame[feature].iloc[i] = result
        
        except:
            outside_search_area.append((i, coordinates[i]))
     
    for i in range(len(outside_search_area)): # for the few coordinates that lie just outside the city
    
        point_district = utilities.closest_to(neighborhoods,outside_search_area[i][1])
    
        result = float(dict(pd.DataFrame(graph.run('match (a:neighborhood) \
                                                where a.name = "{}" return a'.format(point_district)). \
                                                to_table()).iloc[0,0])['zestimate'])
    
    
        demand_frame[feature].iloc[outside_search_area[i][0]] = result
        
    return demand_frame

In [41]:
demand = graph_to_demand_model(graph, demand, "zestimate", business_coordinates, neighborhoods)

In [42]:
demand

,abi,year,employee_size_location,sales_volume_location,year_2016,employee_size_location_2016,sales_volume_location_2016,year_2015,employee_size_location_2015,sales_volume_location_2015,year_established,latitude,longitude,zestimate
0,5780192,2017,45.0,7959.0,2016.0,45.0,12148.0,2015.0,60.0,16198.0,1953.0,41.96843,-87.73453,369466.928527
1,8967192,2017,40.0,9640.0,2016.0,40.0,13420.0,2015.0,40.0,13420.0,1925.0,41.88798,-87.67486,345766.027933
2,9321290,2017,25.0,3853.0,2016.0,25.0,6089.0,2015.0,25.0,3000.0,1940.0,41.86212,-87.69216,174195.314741
3,9323510,2017,20.0,4287.0,2016.0,20.0,6045.0,2015.0,20.0,6045.0,1917.0,41.88546,-87.73210,194151.698144
4,9324856,2017,38.0,9063.0,2016.0,38.0,12638.0,2015.0,38.0,12638.0,1938.0,41.94774,-87.64933,550774.843931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,418290722,2017,52.0,12218.0,2016.0,52.0,19176.0,2015.0,52.0,19176.0,1925.0,41.77312,-87.75264,216874.451473
887,533855128,2017,6.0,1410.0,2016.0,6.0,2212.0,2015.0,6.0,2212.0,1917.0,41.82636,-87.73097,161538.790567
888,655861375,2017,500.0,0.0,2016.0,500.0,NaN,2015.0,500.0,NaN,1917.0,41.88118,-87.63913,508539.775596
889,410866459,2017,3.0,767.0,2016.0,3.0,796.0,2015.0,5.0,1327.0,NaN,41.88171,-87.63086,802738.017279


In [26]:
# Loading average property value for the neighborhood in which the stores is located

demand['neighborhood_avg_property_value'] = np.nan

outside_city = []

for i in range(len(coordinates)):

    point_district = utilities.point_lookup(neighborhoods,coordinates[i])
    
    try:
        result = float(dict(pd.DataFrame(graph.run('match (a:neighborhood) where a.name = "{}" return a'.format(point_district)). \
                            to_table()).iloc[0,0])['zestimate'])
        ## coordinates and df indices should be the same ## 
        demand['neighborhood_avg_property_value'].iloc[i] = result
        
    except:
        outside_city.append((i, coordinates[i]))
     
    
    
for i in range(len(outside_city)): # for the few coordinates that lie just outside the city
    
    point_district = utilities.closest_to(neighborhoods,outside_city[i][1])
    
    result = float(dict(pd.DataFrame(graph.run('match (a:neighborhood) where a.name = "{}" return a'.format(point_district)). \
                            to_table()).iloc[0,0])['zestimate'])
    
    
    demand['neighborhood_avg_property_value'].iloc[outside_city[i][0]] = result
    

In [27]:
# Loading number of property crimes per neighborhood into demand model for each store

demand['neighborhood_property_crimes'] = np.nan

outside_city = []

for i in range(len(coordinates)):

    point_district = utilities.point_lookup(neighborhoods,coordinates[i])
    
    try:
        result = float(dict(pd.DataFrame(graph.run('match (a:neighborhood) where a.name = "{}" return a'.format(point_district)). \
                            to_table()).iloc[0,0])['n_property_crimes'])
        
        ## coordinates and df indices should be the same ## 
        demand['primary_type'].iloc[i] = result
        
    except:
        outside_city.append((i, coordinates[i]))
     
    
for i in range(len(outside_city)): # for the few coordinates that lie just outside the city
    
    point_district = utilities.closest_to(neighborhoods,outside_city[i][1])
    
    result = float(dict(pd.DataFrame(graph.run('match (a:neighborhood) where a.name = "{}" return a'.format(point_district)). \
                            to_table()).iloc[0,0])['primary_type'])
    
    
    demand['neighborhood_property_crimes'].iloc[outside_city[i][0]] = result
    

In [28]:
# Loading average property values of neighborhoods surrounding the neighborhood of the given store

demand['surrounding_neighborhood_avg_property_value'] = np.nan

outside_city = []

for i in range(len(coordinates)):

    point_district = utilities.point_lookup(neighborhoods,coordinates[i])
    
    try:
        result = pd.DataFrame(graph.run('match (a:neighborhood)-[:NEXT_TO]->(b) where a.name = "{}" return b'.format(point_district)). \
                            to_table())

        n_next_door = len(result[0])

        neighboring_means = []

        for j in range(n_next_door):
            neighboring_mean = float(dict(result[0][j])['zestimate'])
            neighboring_means.append(neighboring_mean)
            
        surrounding_mean = np.nanmean(neighboring_means)
        
        ## coordinates and df indices should be the same ## 
        demand['surrounding_neighborhood_avg_property_value'].iloc[i] = surrounding_mean
            
    except:
        outside_city.append((i, coordinates[i]))
     
    
    
for i in range(len(outside_city)): # for the few coordinates that lie just outside the city
    
    point_district = utilities.closest_to(neighborhoods,outside_city[i][1])
    
    result = pd.DataFrame(graph.run('match (a:neighborhood)-[:NEXT_TO]->(b) where a.name = "{}" return b'.format(point_district)). \
                            to_table())

    n_next_door = len(result[0])

    neighboring_means = []

    for i in range(n_next_door):
        neighboring_mean = float(dict(result[0][i])['zestimate'])
        neighboring_means.append(neighboring_mean)
        
    surrounding_mean = np.nanmean(neighboring_means)
    
    demand['surrounding_neighborhood_avg_property_value'].iloc[outside_city[i][0]] = surrounding_mean
    

In [30]:
# Loading number of property crimes of neighborhoods surrounding the neighborhood of the given store

demand['surrounding_neighborhood_property_crimes'] = np.nan

outside_city = []

for i in range(len(coordinates)):

    point_district = utilities.point_lookup(neighborhoods,coordinates[i])

    try:
        result = pd.DataFrame(graph.run('match (a:neighborhood)-[:NEXT_TO]->(b) where a.name = "{}" return b'.format(point_district)). \
                            to_table())

        n_next_door = len(result[0])

        neighboring_means = []

        for j in range(n_next_door):
            neighboring_mean = float(dict(result[0][j])['primary_type'])
            neighboring_means.append(neighboring_mean)
            
        surrounding_mean = np.nanmean(neighboring_means)
        
        ## coordinates and df indices should be the same ## 
        demand['surrounding_neighborhood_property_crimes'].iloc[i] = surrounding_mean
            
    except:
        outside_city.append((i, coordinates[i]))
        
    
for i in range(len(outside_city)): # for the few coordinates that lie just outside the city
    
    point_district = utilities.closest_to(neighborhoods,outside_city[i][1])
    
    result = pd.DataFrame(graph.run('match (a:neighborhood)-[:NEXT_TO]->(b) where a.name = "{}" return b'.format(point_district)). \
                            to_table())

    n_next_door = len(result[0])

    neighboring_means = []

    for i in range(n_next_door):
        neighboring_mean = float(dict(result[0][i])['primary_type'])
        neighboring_means.append(neighboring_mean)
        
    surrounding_mean = np.nanmean(neighboring_means)
    
    demand['surrounding_neighborhood_property_crimes'].iloc[outside_city[i][0]] = surrounding_mean
    

In [32]:
# Least squares model for 'D' component of objective function:

D_demand = demand[(demand['sales_volume_location_2017'].notna()) &
               (demand['sales_volume_location_2016'].notna()) &
               (demand['neighborhood_property_crimes'].notna()) &
               (demand['neighborhood_avg_property_value'].notna()) &
               (demand['surrounding_neighborhood_avg_property_value'].notna()) &
               (demand['surrounding_neighborhood_property_crimes'].notna())]

D_demand_features = ["sales_volume_location_2016","neighborhood_avg_property_value",
                     "neighborhood_property_crimes",
           "surrounding_neighborhood_avg_property_value","surrounding_neighborhood_property_crimes"]

D_X = np.array(D_demand[D_demand_features])
D_y = np.array(D_demand["sales_volume_location_2017"])


# Least squares model for 'L' component of objective functon

L_demand = demand[(demand['sales_volume_location_2017'].notna()) &
               (demand['neighborhood_property_crimes'].notna()) &
               (demand['neighborhood_avg_property_value'].notna()) &
               (demand['surrounding_neighborhood_avg_property_value'].notna()) &
               (demand['surrounding_neighborhood_property_crimes'].notna())]

L_demand_features = ["neighborhood_avg_property_value",
                     "neighborhood_property_crimes",
           "surrounding_neighborhood_avg_property_value","surrounding_neighborhood_property_crimes"]

L_X = np.array(L_demand[L_demand_features])
L_y = np.array(L_demand["sales_volume_location_2017"])




In [33]:
D_lr = LinearRegression()

D_model = D_lr.fit(D_X,D_y)

L_lr = LinearRegression()

L_model = L_lr.fit(L_X,L_y)


In [34]:
D_coef = list(D_model.coef_)
D_coef

[0.8495170614107845,
 -0.0035395772228413148,
 -0.06481868191416973,
 0.0035768328536469702,
 -0.17406478649526722]

In [35]:
L_coef = list(L_model.coef_)
L_coef

[0.0017009471728955767,
 -0.2154568040054697,
 0.005888396027280418,
 -0.5368069460075341]

In [27]:
# save as txt file

D_model_params = [D_demand_features, D_coef]

with open('opt_variables/D_demand_coef.txt', 'w') as model_text:
    for listitem in D_model_params:
        model_text.write('%s\n' % listitem)
        
        # save as txt file

L_model_params = [L_demand_features, L_coef]

with open('opt_variables/L_demand_coef.txt', 'w') as model_text:
    for listitem in L_model_params:
        model_text.write('%s\n' % listitem)